## Libraries

In [1]:
import sys
sys.path.append('../')

In [2]:
import pandas as pd
import geopandas as gdp
import folium
from folium import Choropleth, Circle, Marker, Icon, Map
from folium.plugins import HeatMap, MarkerCluster
import re

from src import CleaningFunctions as cf

## Load Data 

In [3]:
barcelona = pd.read_csv('../Data/GeoDataFrames/gdf_barcelona.csv')
madrid = pd.read_csv('../Data/GeoDataFrames/gdf_madrid.csv')
gijon = pd.read_csv('../Data/GeoDataFrames/gdf_gijon.csv')
bilbao = pd.read_csv('../Data/GeoDataFrames/gdf_bilbao.csv')

## Cleaning dataframes

The category column had inverted quotes. I used the replace to remove them and the strip function to remove the spaces between the words. 

In [4]:
df_madrid = cf.cleaning(madrid, 'category', 'madrid')

../src/CleaningFunctions.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r"[\"\',]", '')


In [5]:
df_barcelona = cf.cleaning(barcelona, 'category', 'barcelona')

../src/CleaningFunctions.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r"[\"\',]", '')


In [6]:
df_gijon = cf.cleaning(gijon, 'category', 'gijon')

../src/CleaningFunctions.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r"[\"\',]", '')


In [7]:
df_bilbao = cf.cleaning(bilbao, 'category', 'bilbao')

../src/CleaningFunctions.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  df[col] = df[col].str.replace(r"[\"\',]", '')


Junto las dos dataframes para poder hacer el ranking

In [8]:
data =df_madrid.append([df_barcelona, df_gijon, df_bilbao])

In [9]:
total_data = data.groupby(["category", "city"]).mean()

In [10]:
total_data.reset_index(drop = False)

,category,city,lat,long,distance
0,Bar,barcelona,41.392523,2.165113,2.153021
1,Bar,bilbao,43.261339,-2.933254,0.874563
2,Bar,gijon,43.537687,-5.658454,5.504250
3,Bar,madrid,40.427318,-3.677368,0.986381
4,Bars,barcelona,41.396544,2.155763,0.854449
5,Basketball Court,barcelona,41.405313,2.168945,2.584342
6,Basketball Court,bilbao,43.260186,-2.948913,2.337619
7,Basketball Court,gijon,43.447403,-5.762851,15.403770
8,Basketball Court,madrid,40.423556,-3.674405,2.173045
9,Preschool,barcelona,41.399940,2.163414,2.955602


In [20]:
# copy the data 
df_min_max_scaled = total_data.copy() 
  
# apply normalization techniques by Column 1 
column = 'distance'
df_min_max_scaled[column] = (df_min_max_scaled[column] - df_min_max_scaled[column].min()) / (df_min_max_scaled[column].max() - df_min_max_scaled[column].min())     
  
df_min_max_scaled

distance
category                      city               
Bar                           barcelona  0.075329
                              bilbao     0.014237
                              gijon      0.235472
                              madrid     0.019580
Bars                          barcelona  0.013276
Basketball Court              barcelona  0.095940
                              bilbao     0.084151
                              gijon      0.708532
                              madrid     0.076286
Preschool                     barcelona  0.113682
                              gijon      1.000000
                              madrid     0.122323
Train                         barcelona  0.013817
                              bilbao     0.013098
                              gijon      0.240693
                              madrid     0.105506
Vegetarian / Vegan Restaurant barcelona  0.008026
                              bilbao     0.000000
                              gijon      0.347436
                              madrid     0.015832

In [11]:
map_2 = Map(location=[40.42955, -3.6793],zoom_start=15)
map_2

In [18]:
parque = folium.Map(location = [40.42955, -3.6793],tooltip="company")

In [19]:
parque

In [22]:
for i,row in data.iterrows():
    #popup distrito
    city = {
        "location" : [row["lat"],row["long"]],
        "tooltip" : row["city"]    
    }
    
    if row["category"] == "Bar":
        icon = Icon(color = "green",
                    prefix = "fa",
                    icon = "glass",
                    icon_color = "black"
        )
    elif row["category"] == "Basketball Court":
        icon = Icon(color = "purple",
                    prefix = "fa",
                    icon = "futbol-o",
                    icon_color = "black"
        )
    elif row["category"] == "Preschool":
        icon = Icon(color = "blue",
                    prefix = "fa",
                    icon = "child",
                    icon_color = "black"
        )       
    elif row["category"] == "Train":
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "train",
                    icon_color = "black"
        )        
    else:
        icon = Icon(color = "red",
                    prefix = "fa",
                    icon = "apple",
                    icon_color = "black"
        )
    Marker(icon = icon).add_to(parque)
map_2

ValueError: Marker location must be assigned when added directly to map.